# SHM Function Selector - Phase 3 Context Menu Test

This notebook tests the Phase 3 context menu functionality of the SHM Function Selector JupyterLab extension.

## Features to Test:

1. **Parameter Detection**: Right-click on function parameter values
2. **Smart Variable Compatibility**: Recommended vs other variables
3. **Professional Context Menu**: Clean interface with type information
4. **Code Modification**: Automatic parameter linking with variable selection

## Instructions:

1. Run the first few cells to create test variables
2. In the SHM function calls below, **right-click on parameter values** (like `data=None` or `fs=1000`)
3. Select variables from the context menu to link them
4. Observe the automatic code modification

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Make sure shmtools is available
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../../'))

try:
    import shmtools
    print("✅ shmtools imported successfully")
except ImportError as e:
    print(f"❌ shmtools not available: {e}")
    print("Run: pip install -e . from the shmtools-python directory")

## Step 1: Create Test Variables

These variables will be detected by the context menu system:

In [ ]:
# Create test data arrays (should be recommended for 'data' parameters)
acceleration_data = np.random.randn(1000, 4)  # 4-channel acceleration data
vibration_signals = np.random.randn(2048, 2)  # 2-channel vibration data
sensor_readings = pd.DataFrame(np.random.randn(500, 3), columns=['x', 'y', 'z'])

print(f"Created acceleration_data: {acceleration_data.shape}")
print(f"Created vibration_signals: {vibration_signals.shape}")
print(f"Created sensor_readings: {sensor_readings.shape}")

In [ ]:
# Create frequency and sampling rate variables (should be recommended for 'fs' parameters)
sampling_frequency = 1000.0  # Hz
fs_accel = 2048.0           # Hz  
data_rate = 500             # Hz
freq_nyquist = 250.0        # Hz

print(f"Sampling rates: {sampling_frequency}, {fs_accel}, {data_rate} Hz")

In [ ]:
# Create order and component variables (should be recommended for model order parameters)
model_order = 15
ar_order = 20
n_components = 5
n_features = 10

print(f"Model parameters: order={model_order}, ar_order={ar_order}")
print(f"Component parameters: n_components={n_components}, n_features={n_features}")

In [ ]:
# Create some other variables that should appear in "Other variables" section
results_dict = {'mae': 0.05, 'rmse': 0.12}
channel_names = ['X', 'Y', 'Z', 'R']
timestamp = '2025-07-21'

print(f"Other variables created: results_dict, channel_names, timestamp")

## Step 2: Test Context Menu on SHM Function Calls

**Instructions**: Right-click on the parameter values in the cells below to test the context menu:

- Right-click on `None` in `data=None` → Should show recommended data arrays
- Right-click on `1000` in `fs=1000` → Should show recommended frequency variables  
- Right-click on `15` in `order=15` → Should show recommended order variables

In [ ]:
# TEST 1: AR Model Parameter Linking
# Right-click on 'None' and 'order=15' to test parameter detection and linking

if 'shmtools' in globals():
    # Right-click on the parameter values below:
    ar_coeffs, residuals = shmtools.ar_model_shm(
        data=None,      # ← RIGHT-CLICK HERE - should show data arrays
        order=15        # ← RIGHT-CLICK HERE - should show order variables
    )
else:
    print("shmtools not available - showing syntax only")
    # ar_coeffs, residuals = shmtools.ar_model_shm(data=None, order=15)

In [ ]:
# TEST 2: PSD Analysis Parameter Linking  
# Right-click on parameter values to test smart compatibility

if 'shmtools' in globals():
    # Right-click on the parameter values below:
    frequencies, psd_values = shmtools.psd_welch_shm(
        data=None,           # ← RIGHT-CLICK HERE - should show acceleration_data, vibration_signals
        fs=1000,            # ← RIGHT-CLICK HERE - should show sampling_frequency, fs_accel  
        nperseg=256         # ← RIGHT-CLICK HERE - should show n_features or other numeric vars
    )
else:
    print("shmtools not available - showing syntax only")
    # frequencies, psd_values = shmtools.psd_welch_shm(data=None, fs=1000, nperseg=256)

In [ ]:
# TEST 3: TODO Comments (should also trigger context menu)
# The extension should detect TODO comments as parameter placeholders

if 'shmtools' in globals():
    # Right-click anywhere in the TODO comment lines:
    filtered_data = shmtools.filter_butterworth_shm(
        data=None,        # TODO: Set data parameter  ← RIGHT-CLICK HERE
        fs=None,          # TODO: Set sampling rate     ← RIGHT-CLICK HERE  
        lowcut=1.0,
        highcut=100.0,
        order=4
    )
else:
    print("shmtools not available - showing syntax only")

In [ ]:
# TEST 4: Multi-line Function Calls
# Test context menu on parameters spanning multiple lines

if 'shmtools' in globals():
    try:
        # Right-click on parameter values in this multi-line call:
        pca_model, explained_variance = shmtools.learn_pca_shm(
            features=None,           # ← RIGHT-CLICK HERE
            n_components=5,          # ← RIGHT-CLICK HERE
            whiten=True,
            random_state=42
        )
    except AttributeError:
        print("Function not available in current shmtools version")
else:
    print("shmtools not available - showing syntax only")

## Step 3: Expected Behavior

When you right-click on parameter values, you should see:

### ✅ **Context Menu Features**:
1. **Header**: "Link parameter: [parameter_name]"
2. **Recommended Variables** (green background):
   - For `data` parameters: `acceleration_data`, `vibration_signals`, `sensor_readings`
   - For `fs` parameters: `sampling_frequency`, `fs_accel`, `data_rate`, `freq_nyquist`
   - For `order`/`n_components` parameters: `model_order`, `ar_order`, `n_components`
3. **Other Variables** (gray background): All other variables
4. **Variable Information**: Type and source cell for each variable

### ✅ **Code Modification**:
- Clicking a variable should replace the parameter value automatically
- TODO comments should be removed
- Success notification should appear

### 🔧 **Toolbar Button**:
- Look for the "🔗 SHM Parameter Linker" button in the notebook toolbar
- Click it for more information about the extension

## Troubleshooting

If the context menu doesn't appear:

1. **Check Browser Console**: Open browser dev tools (F12) and check for errors
2. **Extension Loading**: Look for the "🔗 SHM Parameter Linker" button in the toolbar
3. **Cursor Position**: Make sure you're right-clicking directly on parameter values
4. **JupyterLab Version**: Extension requires JupyterLab 4.0+

### Debug Information:
Run this cell to check if the extension is properly loaded:

In [ ]:
# Check JupyterLab version and extensions
import subprocess
import sys

try:
    # Check JupyterLab version
    result = subprocess.run([sys.executable, '-m', 'jupyter', '--version'], 
                          capture_output=True, text=True)
    print("Jupyter versions:")
    print(result.stdout)
    
    # Check installed extensions
    result = subprocess.run([sys.executable, '-m', 'jupyter', 'labextension', 'list'], 
                          capture_output=True, text=True)
    print("\nInstalled JupyterLab extensions:")
    print(result.stdout)
    
except Exception as e:
    print(f"Error checking versions: {e}")

## Success Criteria

Phase 3 is working correctly if:

- ✅ Right-clicking on parameter values shows the context menu
- ✅ Variables are properly categorized as "Recommended" vs "Other variables"
- ✅ Menu shows variable types and source cells
- ✅ Clicking a variable replaces the parameter value in the code
- ✅ TODO comments are automatically removed
- ✅ Success notifications appear after linking
- ✅ Multi-line function calls work correctly

**Phase 3 Complete!** 🎉 The context menu system provides intelligent parameter linking with a professional interface.